### Code to Preprocess UKB Raw Dataset

In [1]:
# import library
import pyspark
from pyspark.sql import SparkSession

# define data path
path_in = "../../../ukb51139_v2.csv"
path_out = "../../../ukb_subset_v2.csv"

# set up PySpark session
spark = SparkSession.builder.master('local[*]') \
                            .config("spark.driver.memory", "50g") \
                            .appName("UKB").getOrCreate()

22/12/16 15:26:33 WARN Utils: Your hostname, ccrfai resolves to a loopback address: 127.0.1.1; using 163.1.212.155 instead (on interface eno8303)
22/12/16 15:26:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 15:26:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load, Manipulate, Save Dataset

In [4]:
# Load ukb as spark data frame
df = spark.read.csv(path_in, header=True)

# Filter missing blood pressure rows
df = df.filter("(`X4080.2.0` is NOT NULL OR `X93.2.0` is NOT NULL) AND \
                (`X4079.2.0` is NOT NULL OR `X94.2.0` is NOT NULL)")

# Show blood pressure columns
df.select(df["`X4080.2.0`"], df["`X4079.2.0`"], 
          df["`X93.2.0`"], df["`X94.2.0`"]).limit(10).show()

# Save data to file for analysis somewhere else
df.write.csv(path_out)

# register data frame as a sql table to run queries
df.registerTempTable('ukb')

# run SQL queries
#spark.sql('SELECT COUNT(*) FROM ukb').show()
#spark.sql('SELECT COUNT(*) FROM ukb \
#           WHERE `X4080.2.0` > 160 AND `X4079.2.0` > 100').show()

+---------+---------+-------+-------+
|X4080.2.0|X4079.2.0|X93.2.0|X94.2.0|
+---------+---------+-------+-------+
|      116|       58|   null|   null|
|      157|      108|   null|   null|
|      151|       86|   null|   null|
|     null|     null|    139|     82|
|      144|       83|   null|   null|
|      152|       76|   null|   null|
|      145|       75|   null|   null|
|      140|       74|   null|   null|
|      161|       92|   null|   null|
|      112|       62|   null|   null|
+---------+---------+-------+-------+



+--------+
|count(1)|
+--------+
|   46211|
+--------+

22/12/16 15:29:06 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
